<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge



import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

In [3]:
# зафиксируем random state для воспроизводимости результатов
RANDOM_STATE = 42

Будем предсказывать зарплату по описанию вакансии - это задача регрессии!

In [3]:
df = pd.read_csv('salary_data.csv', index_col = [0])

In [4]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,hourly,employer_provided,min_salary,max_salary,avg_salary,company_txt,job_state,same_state,age,python_yn,R_yn,spark,aws,excel
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEducation Required: Bachelor’s degree required, prefe...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1,0,0,53,91,72.0,Tecolote Research\n,NM,0,47,1,0,0,0,1
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe Healthcare Data Scientist position will join our ...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,0,0,63,112,87.5,University of Maryland Medical System\n,MD,0,36,1,0,0,0,0
2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information security company. We are the world's largest provider...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1,0,0,80,90,85.0,KnowBe4\n,FL,1,10,1,0,1,0,1
3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\nDirectorate: Earth & Biological Sciences\n\nDivisi...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewable Energy Lab, Los Alamos National Laboratory",0,0,56,97,76.5,PNNL\n,WA,1,55,1,0,0,0,0
4,Data Scientist,$86K-$143K (Glassdoor est.),"Data Scientist\nAffinity Solutions / Marketing Cloud seeks smart, curious, technically savvy can...",2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",0,0,86,143,114.5,Affinity Solutions\n,NY,1,22,1,0,0,0,1


Посмотрим на распределение целевой переменной. Построим гистограмму распределения вероятностей признака `avg_salary`

In [4]:
plt.hist(df['avg_salary'], bins=70)
plt.grid()
plt.show()

NameError: name 'df' is not defined

Распределение похоже на нормальное. Лучший вариант для линейной регрессии.

Найдем средние/медианные езарплаты в зависимости от профессии. Используем метод `groupby()`

In [6]:
df.groupby('Job Title')['avg_salary'].median()

Job Title
Ag Data Scientist                                      80.5
Analytics - Business Assurance Data Analyst            43.0
Analytics Consultant                                   66.5
Analytics Manager                                      87.5
Analytics Manager - Data Mart                          64.0
                                                      ...  
System and Data Analyst                                59.0
Systems Engineer II - Data Analyst                     62.5
Technology-Minded, Data Professional Opportunities     70.5
VP, Data Science                                      124.5
Web Data Analyst                                      106.0
Name: avg_salary, Length: 264, dtype: float64

In [7]:
df['Job Title'].nunique()

264

Разобъем данные на тренировочные и тестовые. Для обучения будем использовать в качестве признаков только описание вакансии `Job Description`	

In [8]:
X = df['Job Description']
y = df['avg_salary']

In [9]:
%%time
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

CPU times: user 3.13 ms, sys: 2.19 ms, total: 5.31 ms
Wall time: 4.45 ms


Закодируйте столбец `Job Description` при помощи **tf-idf**:

* обучите (метод `fit`) на тренировочных данных `X_train`, а затем примените (`transform`) и к трейну, и к тесту в отдельности.

*  попробуйте подобрать гиперпараметры `min_df`, `max_df`, `ngram_range` у метода для достижения наилучшего качества модели

**Вопрос:** если использовать `min_df = 5`, `max_df=1.0`, `ngram_range=(1,1)`, то сколько столбцов получится в матрице после применения кодировки?

Обучите линейную регрессию `Ridge()` на тренировочных данных и сделайте предсказания на тренировочных и тестовых данных.

**model = Ridge(alpha=1.1)**

In [10]:
tfidf = TfidfVectorizer(max_df=0.7, min_df=5, ngram_range=(1, 2))
tfidf.fit(X_train)
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

KeyboardInterrupt: 

Посчитаем метрику качества предсказания $R^2$. Чем ближе она к 1, тем лучше качество модели.

Посчитайте значение $R^2$ на тренировочных и на тестовых данных.

**Вопрос:** чему равен $R^2$ на тесте?

In [ ]:
model = Ridge(alpha=1.1)

model.fit(X_train_tfidf, y_train)
coefs = sorted(zip(list(np.array(tfidf.get_feature_names_out())[model.coef_ != 0]), model.coef_[model.coef_ != 0]),
               key=lambda x: -x[1])
coefs = pd.DataFrame(coefs, columns=["ngram", "weight"])


r2_train = model.score(X_train_tfidf, y_train)
print(f"R^2 на тренировочных данных: {r2_train}")
r2_test = model.score(X_test_tfidf, y_test)
print(f"R^2 на тестовых данных: {r2_test}")

R^2 на тренировочных данных: 0.8401618156758621
R^2 на тестовых данных: 0.6654780107723559


Потестируйте построенную модель. Посмотрите, как она прогнозирует зарплаты для разных вакансий

In [ ]:
model.predict(tfidf.transform([u"director"]))

array([104.57146156])

In [ ]:
model.predict(tfidf.transform([u"manager"]))

array([92.55765086])

**Вопрос:** посчитайте зарплату "data scientist" при помощи модели.

In [ ]:
model.predict(tfidf.transform([u"data scientist"]))

array([99.15227694])

Посмотрим на веса модели

**Вопрос:** какой признак имеет наибольший положительный вес?


In [ ]:

coefs = sorted(zip(list(np.array(tfidf.get_feature_names_out())[model.coef_ != 0]), model.coef_[model.coef_ != 0]),
               key=lambda x: -x[1])
coefs = pd.DataFrame(coefs, columns=["ngram", "weight"])



In [ ]:
coefs

,ngram,weight
0,data science,29.910814
1,data scientists,27.850719
2,platform,27.681247
3,credit,24.635525
4,gallup,24.170384
...,...,...
15670,data analyst,-22.123619
15671,marketing,-22.566338
15672,hospital,-24.552632
15673,analyst,-26.088860


Закодируйте столбец `Job Description` при помощи мешка слов.

Постройте модель линейной регрессии `Ridge()` на тренировочных данных и сделайте предсказания на тренировочных и тестовых данных, если столбец 
**model = Ridge(alpha=1.1)**

Рассчитайте метрику качества предсказания $R^2$ на тренировочных и на тестовых данных. 

Потестируйте построенную модель. Посмотрите, как она прогнозирует зарплаты для разных вакансий. Сделайте прогноз зарплаты для "data scientist" при помощи построенной модели.


**Вопрос:** Как изменилось качество модели? чему равен $R^2$ на тесте? какой признак имеет наибольший положительный вес?

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cvector = CountVectorizer()

cvector.fit(X_train)

CountVectorizer()

In [ ]:
X_train_cvector = cvector.transform(X_train)
X_test_cvector = cvector.transform(X_test)


In [ ]:
model_cv = Ridge(alpha=1.1)
model_cv.fit(X_train_cvector, y_train)


Ridge(alpha=1.1)

In [ ]:
r2_train = model_cv.score(X_train_cvector, y_train)
print(f"R^2 на тренировочных данных: {r2_train}")
r2_test = model_cv.score(X_test_cvector, y_test)
print(f"R^2 на тестовых данных: {r2_test}")

R^2 на тренировочных данных: 0.9989142548506732
R^2 на тестовых данных: 0.6844225382319408
